# Variáveis

* Agregação por SQ e USO

* Quantidade de contribuintes
* Área = Fração Ideal * Área do terreno (Não deve variar, mas é necessária para TO e CA)
* Área Ocupada = Fração Ideal * Área Ocupada
* Somar Area Construida
* Quantidade de lotes (soma das frações ideiais)
* 'VALOR DO M2 DO TERRENO', 'VALOR DO M2 DE CONSTRUCAO', 'ANO DA CONSTRUCAO CORRIGIDO', 'QUANTIDADE DE PAVIMENTOS', 'TESTADA PARA CALCULO' e 'FATOR DE OBSOLESCENCIA' com média, mediana, max, min e desvio padrão, pode ser describe()

## Carregando e processando dados de cada um dos anos de IPTU

Para cada conjunto de dados de IPTU:

* Carregar os dados
* Obter SQ (Setor/Quadra)
* Adequar os usos aos padroes estabelecidos
* Calcular a área e área ocupada pela fração ideal
* Agregar por SQ e uso e calcular as variáveis
* Anotar o Ano de Exercício
* Adicionar À lista

E por último salvar o resultado da lista com todos os anos

In [1]:
# Carrengando um complemento para visualizar quanto tempo demoram as tarefas

%load_ext autotime

In [2]:
from pathlib import Path
import os
import pandas as pd

In [3]:
# Padrões de nomenclatura de usos

usos = {
    "apartamento em condomínio":"apartamento",
    "cortiço (habitação coletiva subnormal)":"cortiço",
    "escritório/consultório em condomínio (unidade autônoma)":"escritório ou consultório",
    "loja em edifício em condomínio (unidade autônoma)":"loja em edifício em condomínio",
    "residência coletiva, exclusive cortiço (mais de uma residência no lote)":"residência coletiva (mais de uma residência no lote), exclusive cortiço",
    "posto de serviço (combustíveis)":"posto de serviço",
    "estacionamento e garagem, não em condomínio":"garagem (exclusive em prédio em condomínio)",
    "garagem (unidade autônoma) de prédio de garagens":"garagem, em prédio de garagens",
    "garagem (unidade autônoma) em edifício em condomínio de escritórios, consultórios ou misto":"garagem em edifício de escritórios, consultórios ou misto",
    "garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial":"garagem em edifício de uso exclusivamente residencial",
    "outras edificações de uso coletivo, com utilização múltipla":"outras edificações do tipo (uso coletivo), com utilização múltipla",
    "outras edificações de uso comercial, com utilização múltipla":"outras edificações do tipo (uso comércio), com utilização múltipla",
    "outras edificações de uso de serviço, com utilização múltipla":"outras edificações do tipo (uso serviço), com utilização múltipla",
    "outras edificações de uso especial, com utilização múltipla":"outras edificações do tipo (uso especial), com utilização múltipla",
    "prédio com uso exclusivamente residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso exclusivamente residencial",
    "prédio com uso misto, predominância de uso não residencial, não em condomínio":"prédio de escritório, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios) com ou sem loja (predominância comercial)",
    "prédio com uso misto, predominância de uso residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)",
    "prédio de escritório ou consultório, com ou sem lojas, não em condomínio":"prédio de escritório ou consultório, não em condomínio, com ou sem lojas",
    "flat de uso comercial (semelhante a hotel)":"flat, não residencial",
    "flat residencial em condomínio":"flat, residencial"
}

In [4]:
iptus_agg = []

for path in Path('IPTU').rglob('*.zip'):
    print(path.name)
    
    # Carregar dados do arquivo
    # OBS.: O IPTU de 2016 está separado por ',' e não por ';'
    if path.name == "IPTU_2016.zip":
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=',',\
                     decimal=',',\
                     error_bad_lines=False)
    else:
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=';',\
                     decimal=',',\
                     error_bad_lines=False)
            
    # Obter o SQ (Setor e Quadra)
    df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
    df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]
    
    # Normalizando o lote condominial
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'
    
    # Individualizando a testada e pavimentos por lote (SQLC)
    ## TODO
    df_iptu['AREA CONSTRUIDA'] = pd.to_numeric(df_iptu['AREA CONSTRUIDA'], errors ='coerce')
    df_iptu['AREA OCUPADA'] = pd.to_numeric(df_iptu['AREA OCUPADA'], errors ='coerce')
    df_iptu['AREA DO TERRENO'] = pd.to_numeric(df_iptu['AREA DO TERRENO'], errors ='coerce')
    df_iptu['FRACAO IDEAL'] = pd.to_numeric(df_iptu['FRACAO IDEAL'], errors ='coerce')
    df_iptu['QUANTIDADE DE PAVIMENTOS'] = pd.to_numeric(df_iptu['QUANTIDADE DE PAVIMENTOS'], errors ='coerce')
    df_iptu['TESTADA PARA CALCULO'] = pd.to_numeric(df_iptu['TESTADA PARA CALCULO'], errors ='coerce')

    # Adequando variáveis à fração ideal
    df_iptu['area_terreno'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA DO TERRENO']
    df_iptu['area_ocupada'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA OCUPADA']
    
    # Adequar os usos ao padrão estabelecido
    df_iptu.loc[df_iptu['TIPO DE USO DO IMOVEL']\
                .isin(usos),\
                'TIPO DE USO DO IMOVEL'] = \
    df_iptu[df_iptu['TIPO DE USO DO IMOVEL']\
                 .isin(usos)]['TIPO DE USO DO IMOVEL']\
    .apply(lambda x: usos[x])

    # 3. Agregar por SQ e ANO DO EXERCICIO
    iptu_agg = df_iptu[['setor', \
                        'quadra', \
                        'NUMERO DO CONTRIBUINTE', \
                        'ANO DO EXERCICIO',\
                        'TIPO DE USO DO IMOVEL',\
                        'ANO DA CONSTRUCAO CORRIGIDO', \
                        'AREA CONSTRUIDA', \
                        'FRACAO IDEAL',
                        'area_terreno',
                        'area_ocupada',
                        'QUANTIDADE DE PAVIMENTOS',
                        'TESTADA PARA CALCULO']]\
    .groupby(['setor', 'quadra', 'ANO DO EXERCICIO', 'TIPO DE USO DO IMOVEL'])\
    .agg({'NUMERO DO CONTRIBUINTE': 'count',
         'AREA CONSTRUIDA': 'sum',
         'FRACAO IDEAL': 'sum',
         'area_terreno': 'sum',
         'area_ocupada': 'sum',
         'QUANTIDADE DE PAVIMENTOS': 'max',
         'TESTADA PARA CALCULO': 'mean'})\
    .rename(columns = {'NUMERO DO CONTRIBUINTE': 'quantidade_de_contribuintes',
                      'FRACAO IDEAL': 'quantidade_de_lotes',
                      'AREA CONSTRUIDA': 'area_construida',
                      'QUANTIDADE DE PAVIMENTOS': 'maximo_de_pavimentos',
                      'TESTADA PARA CALCULO':'media_testada'})
    
    iptu_agg.to_csv(f'resultados/{path.name[0:-4]}-agregado-sq-uso.csv.gz')

    # adicionando à lista
    iptus_agg.append(iptu_agg)

IPTU_2010.zip


b'Skipping line 2445507: expected 35 fields, saw 43\n'
/home/fernando/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_1996.zip


/home/fernando/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_2007.zip


...

b'Skipping line 2440049: expected 35 fields, saw 43\n'


IPTU_2003.zip


b'Skipping line 2288886: expected 35 fields, saw 43\n'


IPTU_2012.zip


b'Skipping line 2526350: expected 35 fields, saw 43\n'


In [ ]:
iptus_agg[0]

In [ ]:
df_iptu_modelo = pd.read_csv(f'IPTU/IPTU_2010.zip',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=';',\
                     decimal=',',\
                     error_bad_lines=False)

In [ ]:
df_iptu_modelo['FRACAO IDEAL'] = pd.to_numeric(df_iptu_modelo['FRACAO IDEAL'], errors ='coerce')

In [ ]:
df_iptu_modelo['FRACAO IDEAL']

In [ ]:
df_iptu_modelo[df_iptu_modelo['FRACAO IDEAL'] < 1.0].iloc[1]

In [ ]:
df_iptu_modelo.dtypes

In [ ]:
df_iptu_modelo['area_terreno'] = df_iptu_modelo['FRACAO IDEAL'] * df_iptu_modelo['AREA DO TERRENO']
df_iptu_modelo['area_ocupada'] = df_iptu_modelo['FRACAO IDEAL'] * df_iptu_modelo['AREA OCUPADA']

In [ ]:
df_iptu_modelo['TESTADA PARA CALCULO']